# Thermodynamic analysis with synthetic energies

This example shows a **canonical (Boltzmann) ensemble** analysis using the package's **public Python API**.

Physics intuition:
- Each configuration with energy $E_i$ contributes a Boltzmann weight $\exp[-E_i/(k_B T)]$.
- At low temperature, low-energy configurations dominate the probability mass.
- The Helmholtz free energy $F(T) = -k_B T\ln Z$ decreases with temperature because entropy contributes more strongly.

The notebook creates synthetic energies (no external data), computes thermodynamic summaries, and plots:
1. Probability distribution over configurations at fixed temperature.
2. Free energy vs temperature.

In [ ]:
import time
from pathlib import Path
import tempfile

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from gan_mg import analyze_from_csv, sweep_from_csv

k_B_eV_per_K = 8.617333262e-5
rng = np.random.default_rng(7)
start_time = time.perf_counter()

In [ ]:
# Create a synthetic energy landscape with two nearby basins.
n_configs = 140
energies_low = rng.normal(loc=-0.115, scale=0.007, size=n_configs // 2)
energies_high = rng.normal(loc=-0.075, scale=0.010, size=n_configs // 2)
energies_eV = np.concatenate([energies_low, energies_high])

synthetic_df = pd.DataFrame(
    {
        "structure_id": [f"cfg_{i:03d}" for i in range(n_configs)],
        "mechanism": ["synthetic" for _ in range(n_configs)],
        "energy_eV": energies_eV,
    }
)

workdir = Path(tempfile.mkdtemp(prefix="thermo_example_"))
csv_path = workdir / "synthetic_results.csv"
synthetic_df.to_csv(csv_path, index=False)

csv_path, synthetic_df.head()

In [ ]:
# Analyze a single temperature with the public API.
T_fixed = 900.0
single = analyze_from_csv(csv_path, temperature_K=T_fixed)
single

In [ ]:
# Build per-configuration probabilities at fixed temperature.
# (The API returns aggregate observables; probabilities are computed from the same Boltzmann relation.)
beta = 1.0 / (k_B_eV_per_K * T_fixed)
shifted = energies_eV - energies_eV.min()
weights = np.exp(-beta * shifted)
probabilities = weights / weights.sum()

order = np.argsort(energies_eV)
fig, ax = plt.subplots(figsize=(7, 4))
ax.plot(energies_eV[order], probabilities[order], marker="o", ms=3, lw=1)
ax.set_xlabel("Configuration energy (eV)")
ax.set_ylabel("Boltzmann probability at 900 K")
ax.set_title("Probability distribution over synthetic configurations")
ax.grid(alpha=0.25)
plt.show()

In [ ]:
# Sweep temperatures and plot free energy.
temperatures = np.linspace(300.0, 1500.0, 25)
sweep = sweep_from_csv(csv_path, temperatures=temperatures)

T = np.array([row.temperature_K for row in sweep])
F = np.array([row.free_energy_mix_eV for row in sweep])

fig, ax = plt.subplots(figsize=(7, 4))
ax.plot(T, F, color="tab:red", lw=2)
ax.set_xlabel("Temperature (K)")
ax.set_ylabel("Free energy of mixing (eV)")
ax.set_title("Free energy vs temperature for synthetic ensemble")
ax.grid(alpha=0.25)
plt.show()

In [ ]:
elapsed_s = time.perf_counter() - start_time
print(f"Runtime: {elapsed_s:.3f} s")
assert elapsed_s < 10.0, "Example should run in under 10 seconds."